In [23]:
import cv2
import numpy as np
from math import acos, pi, sqrt
from PIL import Image, ImageFilter
import ipywidgets as widgets 
from IPython.display import display
import io

In [24]:
def gaussian1D(sigma):
    a = sigma
    
    if a % 2 == 0: #mask size가 짝수일 경우 1을 키운다
        a+=1
        
    a2 = a/2
    filter_size = np.array(range(-int(a/2),int(a/2)+1))
    result = np.zeros(len(filter_size))
    
    for i in range(len(filter_size)):
        x = i-a2
        result[i] =  float(np.exp(-(x**2)/(2*sigma**2)) / (2*3.14*sigma**2)) #가우시안 공식
    
    total = np.sum(result)
    result/=total
    
    return result

In [25]:
def gaussian2D(sigma):
    # 두 벡터를 외적하여 x,y 2차원 mask를 생성
    result = np.outer(gaussian1D(sigma),gaussian1D(sigma))
    total = np.sum(result)
    result/=total
    
    return result

In [26]:
print(gaussian2D(3))

[[0.09555231 0.10678153 0.10678153]
 [0.10678153 0.11933039 0.11933039]
 [0.10678153 0.11933039 0.11933039]]


In [27]:
def convolution(array, f):
    a = (len(f)-1)//2
    # boundery를 0으로 채운다
    img_pad = np.pad(array, ((a,a),(a,a)),'constant')
    
    #convolution을 위해 2번 뒤집는다
    mask = np.rot90(f)
    mask = np.rot90(mask)
    
    result_img = np.zeros((len(array), len(array[0])))
    result_img = result_img.astype('float32')
    
    for i in range(len(result_img)):
        for j in range(len(result_img[0])):
            result_img[i][j] = np.sum(img_pad[i:i+len(f),j:j+len(f)]*mask)
    
    return result_img

In [28]:
def real_convolution(array, sigma):
    return convolution(array, gaussian2D(sigma))

In [29]:
img = Image.open('Fig0504(i)(salt-pepper-noise).jpg')

img_arr = np.asarray(img)
img_arr = img_arr.astype('float32')

img_result = real_convolution(img_arr, 3)
img_result = img_result.astype('uint8')
img_result = Image.fromarray(img_result)

img_byte_arr = io.BytesIO()
img_result.save(img_byte_arr, format='PNG')
img_byte_arr = img_byte_arr.getvalue()

show_img = widgets.Image(
    value=img_byte_arr,
    format='png',
    width=300,
    height=400,
)
display(show_img)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x00\x00\x00\x00y\x19\xf7…